# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset' #'colab_main_dataset'
# input_dir = os.path.join(home,'image_datasets')
# output_dir = os.path.join(home,'sql_database')
input_dir = os.path.join(home,'Desktop', 'image_datasets')
output_dir = os.path.join(home,'Desktop','sql_database')
database_names=['scface', 'enfsi2015'] # 'lfw', 'scface', 'forenface', 'enfsi', 'enfsi2015' 'xqlfw', 'utkface','chokepoint'
detector_names=['mtcnn', 'mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab



## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[20:52:38] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[20:52:38] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[20:52:38] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[20:52:38] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[20:52:38] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous ve

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2022-11-24 20:52:39.093635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 20:52:39.291895: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-24 20:52:39.920858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/cv2/../../lib64:/home/andrea/Documents/cuda_software/cuda/lib64:/usr/local/cuda-11.2/lib64
2022-11-24 20:52:39.920935: W tensorflow/stream_executor

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

Creating Db file at /home/andrea/Desktop/sql_database/test_video_dataset.db


In [5]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/Desktop/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7fea9c60be50>,
 'databases': [<sql_face.databases.SCFace at 0x7fea9c5cd850>,
 'detector_names': ['mtcnn', 'mtcnn_serfiq'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
database.create_tables(serfiq=serfiq)

Creating images in SCFace: 4160it [00:00, 18005.56it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]2022-11-24 20:52:45.576473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 20:52:45.577027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 20:52:45.577333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 20:52:45.578395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruc

1/1 [==============================] - 0s 203ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [00:09<00:38,  9.52s/it]

1/1 [==============================] - 0s 99ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [00:10<00:12,  4.24s/it]

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [00:11<00:05,  2.88s/it]

1/1 [==============================] - 0s 98ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [00:13<00:02,  2.79s/it]

1/1 [==============================] - 0s 94ms/step


TRIM Creating Cropped Images for detector mtcnn: 100%|██████████| 5/5 [00:14<00:00,  2.90s/it]
TRIM Creating Cropped Images for detector mtcnn_serfiq:   0%|          | 0/5 [00:00<?, ?it/s][20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[20:53:00] 

In [7]:
database.update_tables(attributes_to_update, serfiq = serfiq)

Update cropped images: 0it [00:00, ?it/s]
Computing embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


Computing embeddings:  20%|██        | 1/5 [00:03<00:13,  3.27s/it]

1/1 [==============================] - 0s 28ms/step


Computing embeddings:  40%|████      | 2/5 [00:03<00:04,  1.62s/it]

...

TRIM: Computing TFace quality:  60%|██████    | 3/5 [00:02<00:01,  1.42it/s]

1/1 [==============================] - 0s 18ms/step


TRIM: Computing TFace quality:  80%|████████  | 4/5 [00:03<00:00,  1.05it/s]

1/1 [==============================] - 0s 18ms/step


TRIM: Computing TFace quality: 100%|██████████| 5/5 [00:04<00:00,  1.24it/s]
